<h1> 개인과제 </h1>
<h2> 주피터랩에서 데이터를 입력받으면 3가지 DB에 동시에 CRUD가 되는 시스템을 만들어라 </h2>
<h3> ex) input을 이용하여 <이름:> 홍길동 (엔터), <나이> 33 (엔터), <주소> 해운대구 (엔터) </h3>
<h3> 입력하고, 데이터를 검색하고, 이름을 고르고 수정하고, 삭제 가능하게 하라 </h3>

## 서버 연결

In [23]:
# !pip install mysql-connector-python
# !pip install oracledb 
# !pip install pymongo

### MySQL

In [35]:
import mysql.connector

try:
    mysql_conn = mysql.connector.connect(
        host = "mysql",  # MySQL 서버주소
        user = "root",
        password = "1234"
    )
    if mysql_conn.is_connected():
        print("MySQL 서버에 성공적으로 연결되었습니다.")
except mysql.connector.Error as err:
    print("MySQL 연결 실패 : ", err)
finally:
    if mysql_conn.is_connected():
        print("연결중...")

MySQL 서버에 성공적으로 연결되었습니다.
연결중...


In [36]:
mysql_cursor = mysql_conn.cursor()
que = "CREATE DATABASE IF NOT EXISTS judb"
mysql_cursor.execute(que)
mysql_conn.close()

In [37]:
mysql_conn = mysql.connector.connect(
        host = "mysql",
        user = "root",
        password = "1234",
        database = "judb"    # 사용할 데이터 베이스
    )
mysql_cursor = mysql_conn.cursor()
print("MySQL 연결 성공")

MySQL 연결 성공


In [38]:
# 테이블 생성
query = "CREATE TABLE IF NOT EXISTS students(id INT AUTO_INCREMENT PRIMARY KEY, name VARCHAR(100), age INT, adr VARCHAR(100))"
mysql_cursor.execute(query)
print("테이블 생성완료")

테이블 생성완료


### MongoDB

In [30]:
from pymongo import MongoClient
client = MongoClient("mongodb://mongodb:27017")

try:
    print(client.admin.command('ping'))
    print("MongoDB 서버에 성공적으로 연결되었습니다.")
except Exception as e:
    print("MongoDB 서버 연결 실패 : ", e)

{'ok': 1.0}
MongoDB 서버에 성공적으로 연결되었습니다.


In [31]:
# 사용할 데이터베이스 선택
db = client["mdb"]
print("사용할 데이터베이스 선택됨")

# 사용할 컬렉션 선택
collection = db["students"]
print("사용할 컬렉션 선택됨")

사용할 데이터베이스 선택됨
사용할 컬렉션 선택됨


### Oracle

In [32]:
import oracledb
    # 데이터베이스 연결시도
try:
    oracle_conn = oracledb.connect(
        user = 'pknu',
        password = '1234',
        dsn = 'oracle/XEPDB1'
    )
    print("Oracle 데이터베이스에 성공적으로 연결 되었습니다.")
    oracle_cursor = oracle_conn.cursor()
except Exception as e:
    print("Oralce 데이터베이스 연결 실패 : ", e)

Oracle 데이터베이스에 성공적으로 연결 되었습니다.


In [25]:
query = """
    BEGIN
        EXECUTE IMMEDIATE '
            CREATE TABLE students (
                id NUMBER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
                name VARCHAR2(50),
                age NUMBER,
                adr VARCHAR2(100)
                )';
        EXCEPTION
            WHEN OTHERS THEN
                IF SQLCODE != -955 THEN
                    RAISE;
                END IF;
            END;
"""
oracle_cursor.execute(query)
print("테이블 생성완료")

테이블 생성완료


## 메인코드

In [34]:
str = """
숫자를 골라주세요.
1. 데이터 등록
2. 데이터 읽기
3. 데이터 수정
4. 데이터 삭제
5. 종료
"""
while True:
    print(str)
    inp = input("선택하신 숫자 : ")
    print(inp, "번을 고르셨습니다.")
    if inp == '1':
        print("====== 정보 입력란 ======")
        inp1 = input("이름을 입력하세요")
        inp2 = input("나이를 입력하세요")
        inp3 = input("주소를 입력하세요")
        print("입력된 정보 : ", inp1, inp2, inp3)
        values = (inp1, inp2, inp3)
        # mysql
        sql = "INSERT INTO students (name, age, adr) VALUES (%s, %s, %s)"
        mysql_cursor.execute(sql, values)
        mysql_conn.commit()
        # oracle
        que = "INSERT INTO students(name,age,adr) VALUES (:1, :2, :3)"
        oracle_cursor.execute(que, values)
        oracle_conn.commit()
        # mongodb
        std = {"name":inp1, "age":inp2, "adr":inp3}
        insert_result = collection.insert_one(std)
        # # oracle
        # que = "INSERT INTO students(name,age,adr) VALUES (:1, :2, :3)"
        # cursor.execute(que,values)
        # conn.commit()
        print("==== 데이터 삽입 완료 ====")
    if inp == '2':
        print("==== 데이터 읽어오는 중 ====")
        que = "SELECT * FROM students"
        # mysql
        print("> mysql")
        mysql_cursor.execute(que)
        mysql_students = mysql_cursor.fetchall()
        for std in mysql_students:
            print(std)
        # oracle
        print("> oracle")
        oracle_cursor.execute(que)
        oracle_students = oracle_cursor.fetchall()
        for std in oracle_students:
            print(std)
        # mongodb
        print("> mongodb")
        students = collection.find()
        for std in students:
            print(std)
        print("===== 데이터 출력 완료 =====")
    if inp == '3': 
        print("====== 정보 수정란 ======")
        up_inp1 = input("수정할 사람의 이름을 입력하세요")
        up_inp2 = input("나이를 다시 입력해주세요")
        up_inp3 = input("주소를 다시 입력해주세요")
        # mysql
        sql = "UPDATE students SET age = %s, adr = %s WHERE name = %s"
        values = (up_inp2, up_inp3, up_inp1)
        mysql_cursor.execute(sql, values)
        mysql_conn.commit()
        # oracle
        que = "UPDATE students SET age = :2, adr = :3 WHERE name = :1"
        values = (up_inp2, up_inp3, up_inp1)
        oracle_cursor.execute(que, values)
        oracle_conn.commit()
        # mongodb
        update = collection.update_one({"name":up_inp1}, {'$set':{"age":up_inp2, "adr":up_inp3}})
        print("수정됨", update)
        print ("==== 정보 수정 완료 ====")
    if inp == '4': 
        print("====== 정보 삭제란 ======")
        de_inp = input("삭제할 사람의 이름을 입력하세요")
        # mysql
        sql = "DELETE FROM students WHERE name = %s"
        values = (de_inp,)
        mysql_cursor.execute(sql, values)
        mysql_conn.commit()
        # oracle
        que = "DELETE FROM students WHERE name = :1"
        values = (de_inp,)
        oracle_cursor.execute(que, values)
        oracle_conn.commit()
        # mongodb
        delete = collection.delete_one({"name":de_inp})
        print("==== 데이터 삭제완료 ====")
    if inp == '5':
        print("종료합니다.")
        break


숫자를 골라주세요.
1. 데이터 등록
2. 데이터 읽기
3. 데이터 수정
4. 데이터 삭제
5. 종료



선택하신 숫자 :  2


2 번을 고르셨습니다.
==== 데이터 읽어오는 중 ====
> mysql
(2, '홍길동', 16, '김해')
> oracle
(1, '홍길동', 20, '부산')
> mongodb
{'_id': ObjectId('67f7199a3d21000b5d2f269c'), 'name': '홍길동', 'age': '20', 'adr': '부산'}
===== 데이터 출력 완료 =====

숫자를 골라주세요.
1. 데이터 등록
2. 데이터 읽기
3. 데이터 수정
4. 데이터 삭제
5. 종료



선택하신 숫자 :  3


3 번을 고르셨습니다.
====== 정보 수정란 ======


...

입력된 정보 :  홍길동 27 부산
==== 데이터 삽입 완료 ====

숫자를 골라주세요.
1. 데이터 등록
2. 데이터 읽기
3. 데이터 수정
4. 데이터 삭제
5. 종료



선택하신 숫자 :  2


2 번을 고르셨습니다.
==== 데이터 읽어오는 중 ====
> mysql
(3, '홍길동', 27, '부산')
> oracle
(2, '홍길동', 27, '부산')
> mongodb
{'_id': ObjectId('67f719f03d21000b5d2f269d'), 'name': '홍길동', 'age': '27', 'adr': '부산'}
===== 데이터 출력 완료 =====

숫자를 골라주세요.
1. 데이터 등록
2. 데이터 읽기
3. 데이터 수정
4. 데이터 삭제
5. 종료



선택하신 숫자 :  5


5 번을 고르셨습니다.
종료합니다.
